In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

# Please dont change this--otherwise, you'll get different results from everyone else!
np.random.seed(1547)

DATASET_FOLDER = 'datasets/SA Feedback Surveys_FINAL'

# Intialize df from 2017

In [2]:
# get df for 2017
df_2017 = pd.read_csv(DATASET_FOLDER + '/2017/Student Feedback Surveys-Superview.csv')
df_2017.head(1)

,ID,Location,Track,Week,Rating (Num),Schedule Pacing
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right


In [3]:
# new dataframe series=["id", "year", "location", "track", "week", "rating": categorical, "pace": categorical]
# set the index of the df
df = pd.DataFrame(index=df_2017.ID)
df.index.rename("id", inplace=True)

In [21]:
# set series from the df_2017 to the df
df['year'] = 2017

# location
df['location'] = df_2017.Location

# track
df['track'] = df_2017.Track

# week
def to_integer(value):
    return int(float(value))

def get_int_from_week(value):
    return int(float(value[-1]))

df['week'] = df_2017.Week.apply(get_int_from_week).apply(to_integer).astype(int)

# rating
df['rating'] = pd.Series(df_2017['Rating (Num)'], dtype="category")

# pace
df['pace'] = pd.Series(df_2017['Schedule Pacing'], dtype="category")

In [5]:
df.head()

,year,location,track,week,rating,pace
id,,,,,,
134,2017,Oakland,Apps,1.0,8,A little too fast
36,2017,Santa Clara,Games,3.0,6,Way too slow
117,2017,Oakland,Apps,1.0,8,A little too fast
253,2017,New York City,"Apps, Explorer",1.0,8,A little too fast
350,2017,Santa Clara,"Games, Explorer",3.0,9,Just right


In [22]:
df.dtypes

year           int64
location      object
track         object
week         float64
rating      category
pace        category
dtype: object

# Collecting all CSVs from the 2016 folder

In [7]:
# collect all filenames
from os import walk
import re

DATASET_2016_FOLDER = DATASET_FOLDER + '/2016'

filenames = []
for (dirpath, dirnames, f) in walk(DATASET_2016_FOLDER):
    filenames.extend(f)
    break

# create empty df
df_2016 = pd.DataFrame([], columns=["pace", "rating", "track", "location", "week"])

for a_filename in filenames:
    
    # collect week, location
    m = re.search('Anon Week (\d) Feedback - (.*)\.csv', a_filename)
    if m is None:
        continue
        
    week_number = m.group(1)
    location_title = m.group(2)
    
    # read csv (rating, pace, track?) and append series to the df_2016
    a_df = pd.read_csv(DATASET_2016_FOLDER + '/' + a_filename)
    
    to_append = pd.DataFrame()
    
    try:
        to_append['track'] = a_df['How well is the schedule paced??']
        # display("Track: OK")
    except KeyError:
        # display("Track: error")
        to_append['track'] = np.nan
        
    try:
        to_append['rating'] = a_df['How would you rate your overall satisfaction with the Summer Academy this week?']
        # display("Rating: OK")
    except KeyError:
        # display("Rating: error")
        to_append['rating'] = np.nan
    
    try:
        to_append['pace'] = a_df['How well is the schedule paced?']
        # display("Track: OK")
    except KeyError:
        # display("Track: error")
        to_append['pace'] = np.nan
        
    to_append['week'] = week_number
    to_append['location'] = location_title
        
    df_2016 = df_2016.append(to_append, ignore_index=True, sort=False)
    
df_2016['year'] = 2016
df_2016.head(1)

,pace,rating,track,location,week,year
0,3,3,NaN,Taipei,7,2016


In [8]:
df_2016.pace.value_counts()

3        430
4        163
2         44
5         31
3         20
4         10
1          6
2          4
5          2
#REF!      1
Name: pace, dtype: int64

# Format the 2018 df to match the desired data types

In [9]:
df_2016.week

0      7
1      7
2      7
3      7
4      7
5      7
6      7
7      7
8      7
9      7
10     7
11     6
12     6
13     6
14     6
15     6
16     6
17     6
18     6
19     6
20     6
21     6
22     6
23     6
24     1
25     1
26     1
27     1
28     1
29     1
      ..
897    3
898    3
899    3
900    3
901    3
902    3
903    3
904    3
905    3
906    3
907    3
908    3
909    3
910    3
911    3
912    3
913    3
914    3
915    3
916    3
917    3
918    3
919    3
920    3
921    3
922    3
923    3
924    3
925    3
926    3
Name: week, Length: 927, dtype: object

In [10]:
# Week
df_2016.week = df_2016.week.apply(to_integer)

# rating
df_2016.rating = df_2016.rating.astype('category')

# pace
def pace_from_ints_to_strings(value):
    if value is np.nan:
        return np.nan
    
    if value is not int or int(value) < 1 or int(value) > 5:
        return np.nan
    
    int_value = int(value)
    
    if int_value == 1:
        return "Way too slow"
    elif int_value == 2:
        return "A little too slow"
    elif int_value == 3:
        return "Just right"
    elif int_value == 4:
        return "A little too fast"
    elif int_value == 5:
        return "Way too fast"
        
df_2016.pace = df_2016.pace.apply(pace_from_ints_to_strings).astype('category')

df_2016.dtypes

pace        category
rating      category
track         object
location      object
week           int64
year           int64
dtype: object

# Join two dfs

In [17]:
summer_df = df.append(df_2016, ignore_index=True, sort=False)

In [18]:
summer_df.dtypes

year          int64
location     object
track        object
week        float64
rating       object
pace         object
dtype: object